In [21]:
import pandas as pd
import re
from lib import clean_column_names

In [22]:
import pandas as pd
from collections import defaultdict

def read_wcoi_csv():
    df = pd.read_csv("../data/input/wcoi-llead.csv")
    return df 

def read_wcoi_post():
    df = pd.read_csv("../../post-clean/data/output/post_clean.csv")
    return df 


def jaro_winkler_similarity(s1, s2):
    if not s1 or not s2:
        return 0.0

    match_distance = (max(len(s1), len(s2)) // 2) - 1
    s1_matches = [0] * len(s1)
    s2_matches = [0] * len(s2)
    matches = 0
    transpositions = 0

    for i, ch in enumerate(s1):
        start = max(0, i - match_distance)
        end = min(i + match_distance + 1, len(s2))
        for j in range(start, end):
            if s2[j] == ch and s2_matches[j] == 0:
                s1_matches[i] = 1
                s2_matches[j] = 1
                matches += 1
                break

    if matches == 0:
        return 0.0

    k = 0
    for i, ch in enumerate(s1):
        if s1_matches[i]:
            while s2_matches[k] == 0:
                k += 1
            if s1[i] != s2[k]:
                transpositions += 1
            k += 1

    jaro = ((matches / len(s1)) + (matches / len(s2)) + ((matches - transpositions / 2) / matches)) / 3

    prefix = 0
    for i in range(min(len(s1), len(s2))):
        if s1[i] == s2[i]:
            prefix += 1
        else:
            break
    prefix = min(4, prefix)

    jaro_winkler = jaro + (0.1 * prefix * (1 - jaro))

    return jaro_winkler


def custom_matcher(df):
    results = []
    pair_idx = 0
    
    groups = defaultdict(list)
    for _, row in df.iterrows():
        # Check if first_name and last_name are not NaN
        if pd.notna(row['first_name']) and pd.notna(row['last_name']):
            key = (str(row['first_name'])[0], str(row['last_name'])[0])
        else:
            # If either name is NaN, use a special key
            key = ('', '')
        groups[key].append(row)
    
    for group in groups.values():
        for i, row1 in enumerate(group):
            for j, row2 in enumerate(group[i+1:], start=i+1):
                if row1['data_source'] == row2['data_source']:
                    continue
                
                # Check for NaN values before calculating similarity
                if pd.isna(row1['first_name']) or pd.isna(row1['last_name']) or \
                   pd.isna(row2['first_name']) or pd.isna(row2['last_name']):
                    continue
                
                first_name_sim = jaro_winkler_similarity(str(row1['first_name']), str(row2['first_name']))
                last_name_sim = jaro_winkler_similarity(str(row1['last_name']), str(row2['last_name']))
                sim_score = (first_name_sim + last_name_sim) / 2
                
                if sim_score >= 0.80:
                    result = {
                        'pair_idx': pair_idx,
                        'sim_score': sim_score,
                        'row_key1': row1.name,
                        'first_name1': row1['first_name'],
                        'last_name1': row1['last_name'],
                        'fc1': str(row1['first_name'])[0] if pd.notna(row1['first_name']) else '',
                        'source1': row1['data_source'],
                        'agency1': row1['agency_name'],
                        'wcoi_uid1': row1.get('wcoi_uid', ''),
                        'llead_uid1': row1.get('llead_uid2', ''),
                        'row_key2': row2.name,
                        'first_name2': row2['first_name'],
                        'last_name2': row2['last_name'],
                        'fc2': str(row2['first_name'])[0] if pd.notna(row2['first_name']) else '',
                        'source2': row2['data_source'],
                        'agency2': row2['agency_name'],
                        'wcoi_uid2': row2.get('wcoi_uid', ''),
                        'llead_uid2': row2.get('llead_uid2', '')
                    }
                    
                    # Add filename and page_number for post data
                    if row1['data_source'] == 'post':
                        result['filename1'] = row1['filename']
                        result['page_number1'] = row1['page_number']
                    elif row2['data_source'] == 'post':
                        result['filename2'] = row2['filename']
                        result['page_number2'] = row2['page_number']
                    
                    results.append(result)
                    pair_idx += 1
    
    result_df = pd.DataFrame(results)
    if not result_df.empty:
        result_df = result_df.sort_values('sim_score', ascending=False)
    
    return result_df


wcoi = read_wcoi_csv()
post = read_wcoi_post()
post = post.pipe(clean_column_names)

post = post[["first_name", "last_name", "person_id", "agency_name", "filename", "page_number"]]
post["data_source"] = "post"

wcoi = wcoi[["first_name1", "last_name1", "agency2", "llead_uid2", "wcoi_uid1"]]
wcoi["data_source"] = "wcoi/llead"
wcoi = wcoi.rename(columns={"first_name1": "first_name", "last_name1": "last_name", "agency2": "agency_name"})

combined_df = pd.concat([wcoi, post], ignore_index=True)

matched_df = custom_matcher(combined_df)

matched_df

matched_df.to_csv("../data/output/wcoi_post.csv", index=False)

,pair_idx,sim_score,row_key1,first_name1,last_name1,fc1,source1,agency1,wcoi_uid1,llead_uid1,row_key2,first_name2,last_name2,fc2,source2,agency2,wcoi_uid2,llead_uid2,filename2,page_number2
0,0,1.000000,0,norvel,orazio,n,wcoi/llead,new-orleans-pd,,cee2beb6a6e934880ed126887c2e164d,791,norvel,orazio,n,post,new-orleans-pd,,NaN,LA_Peace_Officer_Reports_7_12_24.pdf,94.0
306,306,1.000000,39,justin,crespo,j,wcoi/llead,new-orleans-pd,,d422e81d64416c4ef33e1d44d50e6433,854,justin,crespo,j,post,new-orleans-pd,,NaN,LA_Peace_Officer_Reports_7_12_24.pdf,114.0
304,304,1.000000,374,harold,lewis,h,wcoi/llead,new-orleans-pd,,25a5055843f0153080281fa925b8cc05,894,harold,lewis,h,post,new-orleans-pd,,NaN,LA_Peace_Officer_Reports_7_12_24_pt.2.pdf,15.0
303,303,1.000000,374,harold,lewis,h,wcoi/llead,new-orleans-pd,,25a5055843f0153080281fa925b8cc05,893,harold,lewis,h,post,new-orleans-pd,,NaN,LA_Peace_Officer_Reports_7_12_24.pdf,128.0
302,302,1.000000,38,harold,lewis,h,wcoi/llead,new-orleans-pd,,40915195a52733053ec9c2c17b7fdcc1,894,harold,lewis,h,post,new-orleans-pd,,NaN,LA_Peace_Officer_Reports_7_12_24_pt.2.pdf,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
565,565,0.802222,363,david,pann david penn,d,wcoi/llead,new-orleans-pd,,95f13c8f3e4ac98fc339c7dfd5a12426,944,david,patrolia,d,post,new-orleans-pd,,NaN,LA_Peace_Officer_Reports_7_12_24_pt.3.pdf,38.0
564,564,0.802222,363,david,pann david penn,d,wcoi/llead,new-orleans-pd,,95f13c8f3e4ac98fc339c7dfd5a12426,943,david,patrolia,d,post,new-orleans-pd,,NaN,LA_Peace_Officer_Reports_7_12_24.pdf,141.0
563,563,0.802222,227,david,pann david penn,d,wcoi/llead,new-orleans-pd,,c53cb1c58918faa7ac27093c598a5e95,944,david,patrolia,d,post,new-orleans-pd,,NaN,LA_Peace_Officer_Reports_7_12_24_pt.3.pdf,38.0
562,562,0.802222,227,david,pann david penn,d,wcoi/llead,new-orleans-pd,,c53cb1c58918faa7ac27093c598a5e95,943,david,patrolia,d,post,new-orleans-pd,,NaN,LA_Peace_Officer_Reports_7_12_24.pdf,141.0
